# Prediction of solar flares from sunspot data
## Data 
The data-files are stored in the `flare.data1` and `flare.data2` files located in the directory `..\data`.
The data was pulled from the [UCI Machine Learning repository](https://archive.ics.uci.edu/ml/datasets.html), who in turn had it donated from Gary Bradshaw. 

## Attribute information
Both datasets are supplied with ten input variables and three classes to be predicted. This problem can be thought of as a multivariate regression problem in that one tries to predict the number of occurences of multiple classes. The input-attributes are as follows: 

   1. Code for class (modified Zurich class)  (A,B,C,D,E,F,H)
   2. Code for largest spot size              (X,R,S,A,H,K)
   3. Code for spot distribution              (X,O,I,C)
   4. Activity                                (1 = reduced, 2 = unchanged)
   5. Evolution                               (1 = decay, 2 = no growth, 3 = growth)
   6. Previous 24 hour flare activity code    (1 = nothing as big as an M1, 2 = one M1, 3 = more activity than one M1)
   7. Historically-complex                    (1 = Yes, 2 = No)
   8. Did region become historically complex  (1 = yes, 2 = no) on this pass across the sun's disk
   9. Area(1 = small, 2 = large)
  10. Area of the largest spot (1 = <=5, 2 = >5)
  
Initially the data is loaded up and preprocessed with the Pandas library. Since three of  the above attributes are categorical they need to be mapped to a numerical value. Pandas provides a method `get_dummies` that accomplishes this in a way that suits our purpose.

## Skewedness
One of the things that makes this dataset fundamentally interesting is that it's pretty _skewed_. From the repository we're informed that the datasets have class distributions as follows: 

**`flare.data1`** :   

|Class    | 0   | 1   | 2   | 4   | Total |
|:-------:|:---:|:---:|:---:|:---:|:-----:|
|C flares |287  | 29  | 7   |  0  | 323   |
|M flares | 291 | 24  | 6   | 2   | 323   |
|X flares | 316 | 7   | 0   |  0  | 323   |

**`flare.data2`** :

|Class  |  0  |  1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | Total |
|:-------:|:---:|:---:|:---:|:---:|:-----:|
| C-class | 884 | 112 | 33 | 20 | 9 | 4 | 3 | 0 | 1 | 1066 |
| M-class | 1030 | 29 | 3 | 2 | 1 | 0 | 1 | 0 | 0 | 1066 |
| X-class | 1061 |  4 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 1066 |

So much of our task will be to find reasonable and clever ways to treat this data. Either by way of sampling or transforming it in some manner. 

In [3]:
#!/usr/bin/env python3
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

from scipy.stats.stats import pearsonr


## Evaulation of results

Before we jump into the nitty-gritty of the data it's important to have considered the way in which the data will be evaluated. 

### $R^2$
The R-squared value of two datasets is a measure of their _correlation_. Useful since it allows a consideration of how closely one dataset follows another. In the function below this coefficient is computed by the python library `scipy`. For our use it will suffice to say that the $R^2$ value is bounded in the interval $[-1,1]$ where values closer to the end points of the interval signify higher degrees of correlation between the datasets.
### P-values


In [4]:
def evaluate_prediction(predicted_vals, y_matr):    
    if isinstance(y_matr, pd.DataFrame):
        y_matr = Y_test.as_matrix()

    C_corr_p = pearsonr(predicted_vals[:,0], y_matr[:,0])
    M_corr_p = pearsonr(predicted_vals[:,1], y_matr[:,1])
    X_corr_p = pearsonr(predicted_vals[:,2], y_matr[:,2])

    for flare_class, corr_p in zip(["C", "M", "X"], [C_corr_p, M_corr_p, X_corr_p]):
        print("Flare class:  {} | Pearson R = {:.3f} | two - tailed p = {:.2e} ".format(
            flare_class,
            corr_p[0],
            corr_p[1]))
    return (C_corr_p, M_corr_p, X_corr_p)
    

In [5]:
df = pd.read_csv("../data/flare.data2", sep  = " ", skiprows  = 1, header = None, 
                 dtype={"0": "category", "1": "category", "2": "category"})
df.columns = ["Class", "largest spot", "spot distr.", "Activity", "Evolution", "Prev 24h act", 
             "Hist. complx.", "Did reg. bec. complx.", "Area", "Area of lrgst. spt.", 
             "C-class - nxt 24h",
             "M-Class - nxt 24h",
             "X-class - nxt 24h"]

df_numerical = pd.get_dummies(df, columns = ["Class", "largest spot", "spot distr."])
df_numerical.head()

,Activity,Evolution,Prev 24h act,Hist. complx.,Did reg. bec. complx.,Area,Area of lrgst. spt.,C-class - nxt 24h,M-Class - nxt 24h,X-class - nxt 24h,...,largest spot_A,largest spot_H,largest spot_K,largest spot_R,largest spot_S,largest spot_X,spot distr._C,spot distr._I,spot distr._O,spot distr._X
0,1,3,1,1,1,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,1,3,1,1,2,1,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,1,3,1,1,2,1,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,1,2,1,1,1,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,1,1,1,1,2,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [6]:
df.shape[0]


1066

## Initial approach
We'll start treating this dataset with naive approaches from machine learning and thought to the sampling of the data. Doing this we'll expect rather poor results

In [7]:
train = df_numerical[:900]
test = df_numerical[900:]

In [8]:
Y_train, Y_test= list(map(lambda x: x.filter(
             ["C-class - nxt 24h",
             "M-Class - nxt 24h",
             "X-class - nxt 24h"],
             axis = 1),
            [train, test]))

X_train, X_test = list(map(lambda x: x.drop(labels = ["C-class - nxt 24h",
             "M-Class - nxt 24h",
             "X-class - nxt 24h"],
            axis = 1), [train, test]))


### Decision Trees
Decision trees are a widely used algorithm both in regression and classification problems. Essentially they construct a hierarchy of if-else questions that lead to a decision (Müller & Guido, 2017). A major complication in modelling with decision trees is avoiding overfitting. A tree that grows arbitrarily complex can trivially fit any dataset by simply having a branch for each datapoint, leading to 100% accuracy for the training-set. But very poor generalization (Müller & Guido, 2017)


In [9]:
tree = DecisionTreeRegressor(random_state = 0)
tree.fit(X_train, Y_train)
predicted_vals = tree.predict(X_test)

a = evaluate_prediction(predicted_vals, Y_test)
    

Flare class:  C | Pearson R = 0.154 | two - tailed p = 4.77e-02 
Flare class:  M | Pearson R = 0.105 | two - tailed p = 1.78e-01 
Flare class:  X | Pearson R = -0.012 | two - tailed p = 8.83e-01 


### Multilayer perceptrons
Resurging in the late 90's multilayer perceptrons have achieved stellar results in both classification and regression. The algorithm is based on applying multiple sets of non-linear transformations to the data to be able to classify or preform regression on complex boundaries. At the heart of the algorithm is the minimization of a cost-function and the iterative updating of the weights by way of back-propagating the error in the outermost layer. Overfitting can be a problem in MLPs, but the consequence is generally less severe than for a decision tree. One primary concern for the MLP is the shape and distribution of the input-data. One of the most applied MLP algorithms "adam" is, for example, very sensitive to this. Generally one should seek to have each input with mean 0 and unit variance (much of the math is matric multiplication, unevenly scaled variables can create large problems).

In [34]:
mlp = MLPRegressor(hidden_layer_sizes = 10, 
                   solver = "lbfgs", 
                   learning_rate = "adaptive", 
                   max_iter = 800,)
mlp.fit(X_train, Y_train)
predicted_vals_mlp = mlp.predict(X_test)

a = evaluate_prediction(predicted_vals_mlp, Y_test)

Flare class:  C | Pearson R = 0.332 | two - tailed p = 1.20e-05 
Flare class:  M | Pearson R = 0.336 | two - tailed p = 9.73e-06 
Flare class:  X | Pearson R = 0.268 | two - tailed p = 4.85e-04 


## Results
As predicted the predictive powers of these models are sub-par, we barely achieved statistically significant results for the C and M class flares and were able to tell nothing useful for the X class. Now let's see if we can get clever. 

# Considerations

### Output representation
In the dataset the number of C, M and X-class flares are integer numbered, but accurately prediciting this integer turns out to be tricky. Instead we might be interested in classifying if the number of flares for a given class is greater than or equal to 0. 

### Sampling
One popular method of error-minimization is some manner of _folding_ the data. Folding generally consists of making sure the model is trained on the whole dataset while still maintaining the ability to generalize. A vital part of this process is  how the data is sampled, a common tactic in computational statistic is to employ **stratified sampling** to ensure that the training and test-sets are balanced in the occurances of anomalies.

## References
A. Müller and S. Guido, _Introduction to Machine Learning with Python_. O'reilly media, 2017